In [1]:
from climsim_utils.data_utils import *

2023-10-18 17:35:27.132587: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
import xarray as xr
import numpy as np

# Loading Data
Link: https://huggingface.co/datasets/LEAP/subsampled_low_res/tree/main

### Loading using huggingface datasets

In [2]:
import datasets
datasets_list = datasets.list_datasets()
print(len(datasets_list))

/var/folders/lc/mjy2_vc10zd7k8yny4sjghbr0000gn/T/ipykernel_6027/216168498.py:2: FutureWarning: list_datasets is deprecated and will be removed in the next major version of datasets. Use 'huggingface_hub.list_datasets' instead.
  datasets_list = datasets.list_datasets()


70759


In [15]:
datasets_list[:5]

['acronym_identification',
 'ade_corpus_v2',
 'adversarial_qa',
 'aeslc',
 'afrikaans_ner_corpus']

In [13]:
'LEAP/subsampled_low_res' in datasets_list ## But too expensive to load

True

In [17]:
ds_builder = datasets.load_dataset_builder('LEAP/subsampled_low_res')

Repo card metadata block was not found. Setting CardData to empty.


In [20]:
ds_builder.config

ParquetConfig(name='default', version=0.0.0, data_dir=None, data_files={NamedSplit('train'): ['hf://datasets/LEAP/subsampled_low_res@7ee4456a9716a9ae4a0ae2b53ec68ba036c2b229/train_input.parquet', 'hf://datasets/LEAP/subsampled_low_res@7ee4456a9716a9ae4a0ae2b53ec68ba036c2b229/train_target.parquet'], NamedSplit('validation'): ['hf://datasets/LEAP/subsampled_low_res@7ee4456a9716a9ae4a0ae2b53ec68ba036c2b229/val_input.parquet', 'hf://datasets/LEAP/subsampled_low_res@7ee4456a9716a9ae4a0ae2b53ec68ba036c2b229/val_target.parquet']}, description=None, batch_size=10000, columns=None, features=None)

### Downloading Files Manually

In [24]:
grid_path = 'ClimSim_Playground/grid_info/ClimSim_low-res_grid-info.nc'
norm_path = 'ClimSim_Playground/preprocessing/normalizations/'

grid_info = xr.open_dataset(grid_path)
input_mean = xr.open_dataset(norm_path + 'inputs/input_mean.nc')
input_max = xr.open_dataset(norm_path + 'inputs/input_max.nc')
input_min = xr.open_dataset(norm_path + 'inputs/input_min.nc')
output_scale = xr.open_dataset(norm_path + 'outputs/output_scale.nc')

data = data_utils(grid_info = grid_info, 
                  input_mean = input_mean, 
                  input_max = input_max, 
                  input_min = input_min, 
                  output_scale = output_scale)

# set variables to V1 subset
data.set_to_v1_vars()

In [32]:
val_input_path

'Data/val_input.npy'

In [30]:
np.load(val_input_path, allow_pickle=True)

UnpicklingError: Failed to interpret file 'Data/val_input.npy' as a pickle

In [28]:
# Change this path to your own
data_path = 'Data/'

train_input_path = data_path + 'train_input.npy'
train_target_path = data_path + 'train_target.npy'
val_input_path = data_path + 'val_input.npy'
val_target_path = data_path + 'val_target.npy'

data.input_train = data.load_npy_file(train_input_path)
data.target_train = data.load_npy_file(train_target_path)
data.input_val = data.load_npy_file(val_input_path)
data.target_val = data.load_npy_file(val_target_path)

ValueError: Cannot load file containing pickled data when allow_pickle=False

### Huggingface URL

In [ ]:
month = '0001-05'


In [3]:
url = "https://huggingface.co/datasets/LEAP/ClimSim_low-res/blob/main/train/0001-02/E3SM-MMF.mli.0001-02-01-00000.nc"

In [18]:
url2 = "https://huggingface.co/datasets/LEAP/ClimSim_low-res/raw/main/train/0001-02/E3SM-MMF.mli.0001-02-01-00000.nc"

In [7]:
import requests
from netCDF4 import Dataset

In [19]:
re = requests.get(url2)

In [22]:
re.content


b'version https://git-lfs.github.com/spec/v1\noid sha256:bd9389d9dfb9c3f34fa8234bf6bd39ff063ebb855dec3eb6662eb9b92fed8133\nsize 1897632\n'

In [23]:
Dataset(url2)

syntax error, unexpected WORD_WORD, expecting SCAN_ATTR or SCAN_DATASET or SCAN_ERROR
context: Entry^ not found


OSError: [Errno -90] NetCDF: file not found: 'https://huggingface.co/datasets/LEAP/ClimSim_low-res/raw/main/train/0001-02/E3SM-MMF.mli.0001-02-01-00000.nc'

In [25]:
xr.open_dataset()

ValueError: did not find a match in any of xarray's currently installed IO backends ['netcdf4', 'scipy', 'pynio']. Consider explicitly selecting one of the installed engines via the ``engine`` parameter, or installing additional IO dependencies, see:
https://docs.xarray.dev/en/stable/getting-started-guide/installing.html
https://docs.xarray.dev/en/stable/user-guide/io.html

### Train OLS Linear Regression

In [ ]:
const_model = data.target_train.mean(axis = 0)

In [ ]:
X = data.input_train
bias_vector = np.ones((X.shape[0], 1))
X = np.concatenate((X, bias_vector), axis=1)

In [ ]:
mlr_weights = np.linalg.inv(X.transpose()@X)@X.transpose()@data.target_train

### Train Own Model

## Evals

In [ ]:
data.set_pressure_grid(data_split = 'val')

In [ ]:
# Constant Prediction
const_pred_val = np.repeat(const_model[np.newaxis, :], data.target_val.shape[0], axis = 0)
print(const_pred_val.shape)

# Multiple Linear Regression
X_val = data.input_val
bias_vector_val = np.ones((X_val.shape[0], 1))
X_val = np.concatenate((X_val, bias_vector_val), axis=1)
mlr_pred_val = X_val@mlr_weights
print(mlr_pred_val.shape)



In [ ]:
# Load your prediction here

# Load predictions into data_utils object
data.model_names = ['const', 'mlr'] # add names of your models here
preds = [const_pred_val, mlr_pred_val] # add your custom predictions here
data.preds_val = dict(zip(data.model_names, preds))

In [ ]:
data.reweight_target(data_split = 'val')
data.reweight_preds(data_split = 'val')

In [ ]:
data.metrics_names = ['MAE', 'RMSE', 'R2', 'bias']
data.create_metrics_df(data_split = 'val')

In [ ]:
# set plotting settings
%config InlineBackend.figure_format = 'retina'
letters = string.ascii_lowercase

# create custom dictionary for plotting
dict_var = data.metrics_var_val
plot_df_byvar = {}
for metric in data.metrics_names:
    plot_df_byvar[metric] = pd.DataFrame([dict_var[model][metric] for model in data.model_names],
                                               index=data.model_names)
    plot_df_byvar[metric] = plot_df_byvar[metric].rename(columns = data.var_short_names).transpose()

# plot figure
fig, axes = plt.subplots(nrows  = len(data.metrics_names), sharex = True)
for i in range(len(data.metrics_names)):
    plot_df_byvar[data.metrics_names[i]].plot.bar(
        legend = False,
        ax = axes[i])
    if data.metrics_names[i] != 'R2':
        axes[i].set_ylabel('$W/m^2$')
    else:
        axes[i].set_ylim(0,1)

    axes[i].set_title(f'({letters[i]}) {data.metrics_names[i]}')
axes[i].set_xlabel('Output variable')
axes[i].set_xticklabels(plot_df_byvar[data.metrics_names[i]].index, \
    rotation=0, ha='center')

axes[0].legend(columnspacing = .9, 
               labelspacing = .3,
               handleheight = .07,
               handlelength = 1.5,
               handletextpad = .2,
               borderpad = .2,
               ncol = 3,
               loc = 'upper right')
fig.set_size_inches(7,8)
fig.tight_layout()